# Gold Layer Advanced Analytics
## Microsoft Fabric Lakehouse - Scout v7 Gold Layer

This notebook creates advanced analytics, market basket analysis, and ML features for the Gold layer.

**Source**: Silver layer tables (Delta format)  
**Target**: Gold analytics tables with ML features  
**Pattern**: Advanced aggregations, market basket, persona scoring

In [ ]:
# Configuration and Setup
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from datetime import datetime, timedelta
import json

print(f"Starting Gold aggregations at: {datetime.now()}")
print("Gold layer: Advanced analytics and ML features")

In [ ]:
# Load Silver Tables
print("📥 Loading Silver layer tables...")

# Main fact tables
transactions_df = spark.table("silver.transactions")
transaction_items_df = spark.table("silver.transaction_items")

# Dimensions
dim_store_df = spark.table("silver.dim_store")
dim_brand_df = spark.table("silver.dim_brand")
dim_category_df = spark.table("silver.dim_category")
dim_date_df = spark.table("silver.dim_date")
dim_time_df = spark.table("silver.dim_time")

# Data quality checks
total_transactions = transactions_df.count()
total_items = transaction_items_df.count()

print(f"📊 Silver data loaded:")
print(f"  - Transactions: {total_transactions:,}")
print(f"  - Transaction Items: {total_items:,}")
print(f"  - Stores: {dim_store_df.count():,}")
print(f"  - Brands: {dim_brand_df.count():,}")

In [ ]:
# Nielsen Category Mappings
print("🏷️ Creating Nielsen category mappings...")

# Enhanced Nielsen mappings with comprehensive coverage
nielsen_mappings = {
    # Tobacco Products
    "tobacco": {
        "l1": "Tobacco Products",
        "l2": "Cigarettes",
        "l3": "Regular Cigarettes",
        "is_tobacco": True,
        "keywords": ["cigarette", "tobacco", "marlboro", "lucky strike", "philip morris"]
    },
    # Laundry & Household
    "laundry": {
        "l1": "Household Care",
        "l2": "Laundry Care",
        "l3": "Fabric Softeners",
        "is_laundry": True,
        "keywords": ["downy", "surf", "ariel", "tide", "fabric"]
    },
    # Food & Beverages
    "food": {
        "l1": "Food & Beverages",
        "l2": "Packaged Food",
        "l3": "Snacks",
        "is_tobacco": False,
        "keywords": ["snack", "food", "beverage", "drink"]
    },
    # Personal Care
    "personal_care": {
        "l1": "Health & Beauty",
        "l2": "Personal Care",
        "l3": "Skin Care",
        "is_tobacco": False,
        "keywords": ["shampoo", "soap", "toothpaste", "cosmetic"]
    }
}

# Convert to DataFrame for joins
nielsen_rows = []
for key, mapping in nielsen_mappings.items():
    for keyword in mapping["keywords"]:
        nielsen_rows.append({
            "category_key": key,
            "keyword": keyword.lower(),
            "nielsen_l1": mapping["l1"],
            "nielsen_l2": mapping["l2"],
            "nielsen_l3": mapping["l3"],
            "is_tobacco": mapping["is_tobacco"],
            "is_laundry": mapping.get("is_laundry", False)
        })

nielsen_df = spark.createDataFrame(nielsen_rows)
nielsen_df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.nielsen_mappings")

print(f"✅ Nielsen mappings created: {len(nielsen_rows)} keyword mappings")

In [ ]:
# Market Basket Analysis
print("🛒 Performing market basket analysis...")

# Item co-occurrence analysis
basket_analysis = transaction_items_df \
    .groupBy("canonical_tx_id") \
    .agg(F.collect_list("sku").alias("items_in_basket")) \
    .withColumn("basket_size", F.size("items_in_basket")) \
    .filter(F.col("basket_size") >= 2)  # Only multi-item baskets

# Generate item pairs for association rules
from itertools import combinations
from pyspark.sql import Row

def generate_pairs(items):
    """Generate all pairs from a list of items"""
    if len(items) < 2:
        return []
    return [Row(item_a=str(pair[0]), item_b=str(pair[1])) for pair in combinations(sorted(items), 2)]

# UDF for pair generation
from pyspark.sql.types import ArrayType, StructType, StructField
pair_schema = ArrayType(StructType([
    StructField("item_a", StringType(), True),
    StructField("item_b", StringType(), True)
]))

generate_pairs_udf = F.udf(generate_pairs, pair_schema)

# Generate item pairs and calculate support
item_pairs = basket_analysis \
    .withColumn("pairs", generate_pairs_udf("items_in_basket")) \
    .select("canonical_tx_id", F.explode("pairs").alias("pair")) \
    .select("canonical_tx_id", 
            F.col("pair.item_a").alias("item_a"),
            F.col("pair.item_b").alias("item_b"))

# Calculate association metrics
total_baskets = basket_analysis.count()

association_rules = item_pairs \
    .groupBy("item_a", "item_b") \
    .agg(F.count("*").alias("co_occurrence_count")) \
    .withColumn("support", F.col("co_occurrence_count") / total_baskets) \
    .filter(F.col("support") >= 0.01)  # Minimum 1% support

# Add confidence and lift calculations
item_support = transaction_items_df \
    .select("sku", "canonical_tx_id") \
    .distinct() \
    .groupBy("sku") \
    .agg(F.count("*").alias("item_count")) \
    .withColumn("item_support", F.col("item_count") / total_baskets)

association_rules_final = association_rules \
    .join(item_support.alias("a"), F.col("item_a") == F.col("a.sku")) \
    .join(item_support.alias("b"), F.col("item_b") == F.col("b.sku")) \
    .withColumn("confidence", F.col("support") / F.col("a.item_support")) \
    .withColumn("lift", F.col("support") / (F.col("a.item_support") * F.col("b.item_support"))) \
    .select("item_a", "item_b", "support", "confidence", "lift", "co_occurrence_count") \
    .filter(F.col("lift") > 1.0)  # Only positive associations

# Save market basket results
association_rules_final.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.market_basket_rules")

basket_summary = association_rules_final.agg(
    F.count("*").alias("total_rules"),
    F.max("lift").alias("max_lift"),
    F.avg("confidence").alias("avg_confidence")
).collect()[0]

print(f"📊 Market basket analysis complete:")
print(f"  - Association rules: {basket_summary['total_rules']:,}")
print(f"  - Max lift: {basket_summary['max_lift']:.2f}")
print(f"  - Avg confidence: {basket_summary['avg_confidence']:.2f}")

In [ ]:
# Customer Analytics and Personas
print("👤 Creating customer analytics and persona features...")

# Customer transaction summary
customer_features = transactions_df \
    .filter(F.col("facial_id").isNotNull()) \
    .groupBy("facial_id", "customer_age", "customer_gender") \
    .agg(
        F.count("*").alias("transaction_count"),
        F.sum("transaction_value").alias("total_spent"),
        F.avg("transaction_value").alias("avg_transaction_value"),
        F.avg("basket_size").alias("avg_basket_size"),
        F.countDistinct("store_id").alias("store_variety"),
        F.min("transaction_date").alias("first_visit"),
        F.max("transaction_date").alias("last_visit"),
        F.collect_set("emotional_state").alias("emotional_states"),
        F.avg("conversation_score").alias("avg_conversation_score")
    ) \
    .withColumn("customer_lifespan_days", 
                F.datediff("last_visit", "first_visit")) \
    .withColumn("visit_frequency", 
                F.when(F.col("customer_lifespan_days") > 0,
                      F.col("transaction_count") / F.col("customer_lifespan_days"))
                .otherwise(0))

# Add spending categories
spending_percentiles = customer_features.approxQuantile("total_spent", [0.33, 0.66], 0.01)
low_spender_threshold = spending_percentiles[0]
high_spender_threshold = spending_percentiles[1]

customer_profiles = customer_features \
    .withColumn("spending_category",
                F.when(F.col("total_spent") <= low_spender_threshold, "Low")
                .when(F.col("total_spent") <= high_spender_threshold, "Medium")
                .otherwise("High")) \
    .withColumn("loyalty_score",
                F.when(F.col("customer_lifespan_days") >= 30, 3)
                .when(F.col("customer_lifespan_days") >= 7, 2)
                .otherwise(1)) \
    .withColumn("engagement_score",
                (F.col("avg_conversation_score") * 0.4 +
                 F.col("visit_frequency") * 100 * 0.3 +
                 F.col("store_variety") * 0.3).cast("float"))

# Save customer profiles
customer_profiles.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.customer_profiles")

customer_summary = customer_profiles.agg(
    F.count("*").alias("total_customers"),
    F.avg("transaction_count").alias("avg_transactions_per_customer"),
    F.avg("total_spent").alias("avg_spend_per_customer")
).collect()[0]

print(f"📊 Customer analytics complete:")
print(f"  - Unique customers: {customer_summary['total_customers']:,}")
print(f"  - Avg transactions per customer: {customer_summary['avg_transactions_per_customer']:.1f}")
print(f"  - Avg spend per customer: {customer_summary['avg_spend_per_customer']:.2f}")

In [ ]:
# Advanced Store Analytics
print("🏪 Creating advanced store analytics...")

# Store performance metrics
store_performance = transactions_df \
    .join(dim_store_df, "store_id") \
    .groupBy("store_id", "store_name", "region_name", "province_name", 
             "municipality_name", "latitude", "longitude") \
    .agg(
        F.count("*").alias("transaction_count"),
        F.sum("transaction_value").alias("total_revenue"),
        F.avg("transaction_value").alias("avg_transaction_value"),
        F.countDistinct("facial_id").alias("unique_customers"),
        F.avg("basket_size").alias("avg_basket_size"),
        F.countDistinct("transaction_date").alias("operating_days"),
        F.avg("conversation_score").alias("avg_conversation_score"),
        F.min("transaction_date").alias("first_transaction"),
        F.max("transaction_date").alias("last_transaction")
    ) \
    .withColumn("revenue_per_day", 
                F.col("total_revenue") / F.col("operating_days")) \
    .withColumn("transactions_per_day", 
                F.col("transaction_count") / F.col("operating_days")) \
    .withColumn("customer_retention", 
                F.col("unique_customers") / F.col("transaction_count"))

# Add store performance categories
revenue_percentiles = store_performance.approxQuantile("total_revenue", [0.25, 0.5, 0.75], 0.01)

store_analytics = store_performance \
    .withColumn("performance_tier",
                F.when(F.col("total_revenue") >= revenue_percentiles[2], "Top")
                .when(F.col("total_revenue") >= revenue_percentiles[1], "High")
                .when(F.col("total_revenue") >= revenue_percentiles[0], "Medium")
                .otherwise("Low")) \
    .withColumn("efficiency_score",
                (F.col("revenue_per_day") / 1000 * 0.4 +
                 F.col("avg_conversation_score") * 0.3 +
                 F.col("customer_retention") * 100 * 0.3).cast("float"))

# Save store analytics
store_analytics.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.store_analytics")

print(f"✅ Store analytics created for {store_analytics.count():,} stores")

In [ ]:
# Time-based Analytics
print("⏰ Creating time-based analytics...")

# Hourly patterns
hourly_patterns = transactions_df \
    .join(dim_time_df, "time_key") \
    .groupBy("hour_24", "time_of_day_category", "business_time_period") \
    .agg(
        F.count("*").alias("transaction_count"),
        F.sum("transaction_value").alias("total_revenue"),
        F.avg("transaction_value").alias("avg_transaction_value"),
        F.countDistinct("facial_id").alias("unique_customers"),
        F.avg("conversation_score").alias("avg_conversation_score")
    ) \
    .withColumn("revenue_per_transaction", 
                F.col("total_revenue") / F.col("transaction_count"))

# Daily patterns with day of week
daily_patterns = transactions_df \
    .join(dim_date_df, "date_key") \
    .groupBy("day_of_week", "day_name", "is_weekend", "is_holiday") \
    .agg(
        F.count("*").alias("transaction_count"),
        F.sum("transaction_value").alias("total_revenue"),
        F.avg("transaction_value").alias("avg_transaction_value"),
        F.countDistinct("facial_id").alias("unique_customers")
    ) \
    .withColumn("revenue_index", 
                F.col("total_revenue") / F.avg("total_revenue").over(Window.partitionBy()))

# Monthly trends
monthly_trends = transactions_df \
    .join(dim_date_df, "date_key") \
    .groupBy("year", "month", "month_name", "quarter") \
    .agg(
        F.count("*").alias("transaction_count"),
        F.sum("transaction_value").alias("total_revenue"),
        F.countDistinct("facial_id").alias("unique_customers"),
        F.countDistinct("store_id").alias("active_stores")
    ) \
    .withColumn("month_year", F.concat(F.col("year"), F.lit("-"), 
                                       F.lpad(F.col("month"), 2, "0")))

# Save time analytics
hourly_patterns.write.mode("overwrite").saveAsTable("gold.hourly_patterns")
daily_patterns.write.mode("overwrite").saveAsTable("gold.daily_patterns")
monthly_trends.write.mode("overwrite").saveAsTable("gold.monthly_trends")

print(f"✅ Time-based analytics created:")
print(f"  - Hourly patterns: {hourly_patterns.count()} hours")
print(f"  - Daily patterns: {daily_patterns.count()} day types")
print(f"  - Monthly trends: {monthly_trends.count()} months")

In [ ]:
# Product Analytics with Nielsen Integration
print("📦 Creating product analytics with Nielsen integration...")

# Enhanced product performance with Nielsen categories
product_performance = transaction_items_df \
    .join(transactions_df.select("canonical_tx_id", "transaction_date", "store_id", "facial_id"), 
          "canonical_tx_id") \
    .groupBy("sku", "item_brand", "item_category", "nielsen_l1", "nielsen_l2", "nielsen_l3") \
    .agg(
        F.count("*").alias("transaction_frequency"),
        F.sum("item_qty").alias("total_quantity_sold"),
        F.sum("item_total").alias("total_revenue"),
        F.avg("item_unit_price").alias("avg_unit_price"),
        F.countDistinct("canonical_tx_id").alias("unique_transactions"),
        F.countDistinct("store_id").alias("store_presence"),
        F.countDistinct("facial_id").alias("unique_customers"),
        F.sum(F.when(F.col("is_substitution") == True, 1).otherwise(0)).alias("substitution_count")
    ) \
    .withColumn("avg_quantity_per_transaction", 
                F.col("total_quantity_sold") / F.col("unique_transactions")) \
    .withColumn("revenue_per_unit", 
                F.col("total_revenue") / F.col("total_quantity_sold")) \
    .withColumn("substitution_rate", 
                F.col("substitution_count") / F.col("transaction_frequency"))

# Add performance tiers
revenue_percentiles = product_performance.approxQuantile("total_revenue", [0.2, 0.4, 0.6, 0.8], 0.01)

product_analytics = product_performance \
    .withColumn("performance_tier",
                F.when(F.col("total_revenue") >= revenue_percentiles[3], "A")
                .when(F.col("total_revenue") >= revenue_percentiles[2], "B")
                .when(F.col("total_revenue") >= revenue_percentiles[1], "C")
                .when(F.col("total_revenue") >= revenue_percentiles[0], "D")
                .otherwise("E")) \
    .withColumn("velocity_score",
                (F.col("transaction_frequency") * 0.4 +
                 F.col("total_quantity_sold") * 0.3 +
                 F.col("store_presence") * 0.3).cast("float"))

# Save product analytics
product_analytics.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.product_analytics")

print(f"✅ Product analytics created for {product_analytics.count():,} SKUs")

In [ ]:
# ML Feature Engineering for Predictive Analytics
print("🤖 Creating ML features for predictive analytics...")

# Customer segmentation features
ml_customer_features = customer_profiles \
    .select(
        "facial_id",
        "customer_age",
        "transaction_count",
        "total_spent",
        "avg_transaction_value",
        "avg_basket_size",
        "store_variety",
        "visit_frequency",
        "loyalty_score",
        "engagement_score"
    ) \
    .filter(F.col("facial_id").isNotNull()) \
    .fillna(0)

# Prepare features for clustering
feature_cols = ["customer_age", "transaction_count", "total_spent", 
                "avg_transaction_value", "avg_basket_size", "store_variety", 
                "visit_frequency", "engagement_score"]

# Assemble feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
ml_features_df = assembler.transform(ml_customer_features)

# Scale features
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(ml_features_df)
scaled_df = scaler_model.transform(ml_features_df)

# K-means clustering for customer segmentation
kmeans = KMeans(featuresCol="scaled_features", predictionCol="cluster", k=5, seed=42)
kmeans_model = kmeans.fit(scaled_df)
clustered_df = kmeans_model.transform(scaled_df)

# Evaluate clustering
evaluator = ClusteringEvaluator(featuresCol="scaled_features", predictionCol="cluster")
silhouette_score = evaluator.evaluate(clustered_df)

# Add cluster interpretation
cluster_summary = clustered_df \
    .groupBy("cluster") \
    .agg(
        F.count("*").alias("cluster_size"),
        F.avg("total_spent").alias("avg_spent"),
        F.avg("transaction_count").alias("avg_transactions"),
        F.avg("engagement_score").alias("avg_engagement")
    ) \
    .withColumn("cluster_label",
                F.when(F.col("avg_spent") > 1000, "High Value")
                .when(F.col("avg_transactions") > 5, "Frequent")
                .when(F.col("avg_engagement") > 5, "Engaged")
                .otherwise("Occasional"))

# Join cluster labels back
ml_customer_segments = clustered_df \
    .join(cluster_summary.select("cluster", "cluster_label"), "cluster") \
    .select("facial_id", "cluster", "cluster_label", *feature_cols)

# Save ML features and segments
ml_customer_segments.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.customer_segments_ml")

cluster_summary.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.cluster_summary")

print(f"🤖 ML features created:")
print(f"  - Customer segments: {ml_customer_segments.count():,} customers")
print(f"  - Silhouette score: {silhouette_score:.3f}")
print(f"  - Clusters: {cluster_summary.count()} segments")

In [ ]:
# Data Quality Summary and Validation
print("📊 Gold Layer Data Quality Summary")
print("=" * 50)

# Check all Gold tables
gold_tables = [
    "gold.nielsen_mappings",
    "gold.market_basket_rules",
    "gold.customer_profiles",
    "gold.store_analytics",
    "gold.hourly_patterns",
    "gold.daily_patterns",
    "gold.monthly_trends",
    "gold.product_analytics",
    "gold.customer_segments_ml",
    "gold.cluster_summary"
]

gold_summary = []

for table in gold_tables:
    try:
        df = spark.table(table)
        row_count = df.count()
        col_count = len(df.columns)
        
        gold_summary.append({
            "table": table,
            "rows": row_count,
            "columns": col_count,
            "status": "✅ Success"
        })
        
        print(f"{table}: {row_count:,} rows, {col_count} columns")
        
    except Exception as e:
        gold_summary.append({
            "table": table,
            "rows": 0,
            "columns": 0,
            "status": f"❌ Error: {str(e)}"
        })
        
        print(f"{table}: ❌ Error - {str(e)}")

# Create summary DataFrame
summary_df = spark.createDataFrame(gold_summary)
summary_df.show(truncate=False)

# Save Gold layer metadata
gold_metadata = {
    "processing_id": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "processing_timestamp": datetime.now().isoformat(),
    "layer": "gold",
    "tables_processed": gold_summary,
    "total_rows_created": sum([item["rows"] for item in gold_summary]),
    "ml_features": {
        "customer_segmentation": True,
        "market_basket_analysis": True,
        "nielsen_integration": True,
        "time_series_features": True
    },
    "analytics_capabilities": {
        "customer_analytics": True,
        "store_performance": True,
        "product_intelligence": True,
        "temporal_patterns": True
    },
    "status": "completed"
}

# Save metadata
metadata_df = spark.createDataFrame([gold_metadata])
metadata_df.write \
    .mode("append") \
    .saveAsTable("gold.processing_metadata")

print("\n" + "=" * 50)
print(f"🎉 Gold layer processing completed successfully!")
print(f"📊 Total rows created: {gold_metadata['total_rows_created']:,}")
print(f"🤖 ML features: Customer segmentation, Market basket, Nielsen integration")
print(f"📈 Analytics: Customer, Store, Product, Time-series")
print(f"⏰ Completed at: {datetime.now()}")
print("\nNext step: Use Gold tables in Warehouse for Power BI")
print("=" * 50)